<a href="https://colab.research.google.com/github/Linoh12/LearningLSTM/blob/main/LSTM_Lincoln.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import

In [4]:
!pip install scikit-learn-intelex

In [5]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torchvision

import sklearn
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

from sklearnex import patch_sklearn
patch_sklearn()

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

from pathlib import Path

torch.__version__

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


'2.0.1+cu118'

In [6]:
fp = 'raw_data.csv'
data = pd.read_csv(fp)
data

,SenderID,ReceiverID,Receiver_XVelocity,Timestamp,Pathloss,PropDelay,Distance,Sender_XVelocity,Sender_YVelocity,Receiver_YVelocity
0,1,2,-3.000000e-08,0.000006,72.6660,1.400000e-07,42.107007,-3.000000e-08,-3.000000e-08,-4.000000e-08
1,1,3,-3.000000e-08,0.000006,71.0602,1.170000e-07,35.000000,-3.000000e-08,-3.000000e-08,-3.000000e-08
2,1,4,-3.000000e-08,0.000006,73.6697,1.580000e-07,47.265209,-3.000000e-08,-3.000000e-08,-3.000000e-08
3,1,5,-3.000000e-08,0.000006,71.8491,1.280000e-07,38.327536,-3.000000e-08,-3.000000e-08,-3.000000e-08
4,1,6,-3.000000e-08,0.000006,75.8679,2.030000e-07,60.876925,-3.000000e-08,-3.000000e-08,-4.000000e-08
...,...,...,...,...,...,...,...,...,...,...
49995,14,17,-4.959106e-01,81.240209,69.7883,1.010000e-07,30.232433,-4.959106e-01,2.479553e+00,2.479553e+00
49996,14,18,-4.959106e-01,81.240209,65.4033,6.100000e-08,18.248288,-4.959106e-01,2.479553e+00,2.479553e+00
49997,14,19,-5.000000e-08,81.240209,90.7695,1.128000e-06,338.479901,-4.959106e-01,2.479553e+00,-9.000000e-08
49998,14,20,-5.000000e-08,81.240209,90.0503,1.039000e-06,311.579745,-4.959106e-01,2.479553e+00,-9.000000e-08


In [7]:
# Applying Standard Scalar to normalize data
ID_features = ['SenderID','ReceiverID']
scaled_data = pd.DataFrame(StandardScaler().fit_transform(data.drop(ID_features,axis=1)), columns=data.drop(ID_features,axis=1).columns)
scaled_data = pd.concat((data[ID_features], scaled_data), axis=1)
scaled_data

,SenderID,ReceiverID,Receiver_XVelocity,Timestamp,Pathloss,PropDelay,Distance,Sender_XVelocity,Sender_YVelocity,Receiver_YVelocity
0,1,2,-0.415269,-2.109152,-1.272443,-1.232738,-1.232222,-0.575348,-0.725865,-0.550458
1,1,3,-0.415269,-2.109152,-1.427426,-1.264487,-1.264923,-0.575348,-0.725865,-0.550458
2,1,4,-0.415269,-2.109152,-1.175571,-1.207890,-1.208487,-0.575348,-0.725865,-0.550458
3,1,5,-0.415269,-2.109152,-1.351286,-1.249303,-1.249612,-0.575348,-0.725865,-0.550458
4,1,6,-0.415269,-2.109152,-0.963413,-1.145772,-1.145856,-0.575348,-0.725865,-0.550458
...,...,...,...,...,...,...,...,...,...,...
49995,14,17,-0.539460,2.064079,-1.550183,-1.286574,-1.286861,-0.681885,0.091776,0.379528
49996,14,18,-0.539460,2.064079,-1.973399,-1.341790,-1.342003,-0.681885,0.091776,0.379528
49997,14,19,-0.415269,2.064079,0.474808,0.131111,0.131483,-0.681885,0.091776,-0.550458
49998,14,20,-0.415269,2.064079,0.405394,0.008254,0.007707,-0.681885,0.091776,-0.550458


In [8]:
# pairs = [{row[0],row[1]} for row in scaled_data[['SenderID','ReceiverID']].values]
# # len(np.unique(pairs, axis=1))
# pairs

In [9]:
# # use np.arange instead of using this np.unique
# IDs = np.unique(scaled_data[['SenderID', 'ReceiverID']])
# possible_pairs = []
# for i,v in enumerate(IDs):
#   temp = list(IDs)
#   temp.pop(i)
#   for choose in temp:
#     possible_pairs.append({v,choose})
# # possible_pairs = pd.Series(possible_pairs, name='pair')

# possible_pairs[:5]
# possible_pairs

In [10]:
possible_pairs = np.unique(pd.Series([{x+1,y+1} if x!=y else None for x in range(26) for y in range(26)]).dropna().values).tolist()
possible_pairs

[{1, 2},
 {4, 18},
 {5, 18},
 {6, 18},
 {7, 18},
 {8, 18},
 {9, 18},
 {10, 18},
 {3, 18},
 {11, 18},
 {13, 18},
 {14, 18},
 {15, 18},
 {16, 18},
 {17, 18},
 {18, 19},
 {18, 20},
 {12, 18},
 {18, 21},
 {2, 18},
 {17, 26},
 {9, 17},
 {10, 17},
 {11, 17},
 {12, 17},
 {13, 17},
 {14, 17},
 {15, 17},
 {1, 18},
 {16, 17},
 {17, 19},
 {17, 20},
 {17, 21},
 {17, 22},
 {17, 23},
 {17, 24},
 {17, 25},
 {17, 18},
 {18, 22},
 {18, 23},
 {18, 24},
 {19, 21},
 {19, 22},
 {19, 23},
 {19, 24},
 {19, 25},
 {19, 26},
 {1, 20},
 {19, 20},
 {2, 20},
 {4, 20},
 {5, 20},
 {6, 20},
 {7, 20},
 {8, 20},
 {9, 20},
 {10, 20},
 {3, 20},
 {18, 19},
 {17, 19},
 {16, 19},
 {18, 25},
 {18, 26},
 {1, 19},
 {2, 19},
 {3, 19},
 {4, 19},
 {5, 19},
 {6, 19},
 {7, 19},
 {8, 19},
 {9, 19},
 {10, 19},
 {11, 19},
 {12, 19},
 {13, 19},
 {14, 19},
 {15, 19},
 {8, 17},
 {11, 20},
 {7, 17},
 {5, 17},
 {14, 24},
 {14, 25},
 {14, 26},
 {1, 15},
 {2, 15},
 {3, 15},
 {4, 15},
 {14, 23},
 {5, 15},
 {7, 15},
 {8, 15},
 {9, 15},
 {10, 1

In [11]:
def make_negatives(possible_pairs, data, random_state, negative_only = True):
  """
  possible_pairs is the total permutations possible between SenderID and RecieverID
  data is the dataset for a given timestamp
  random_state sets the random seed using np.random.seed()
  """

  #Random Seed
  np.random.seed(random_state)
  rng = np.random.default_rng()

  #Remapping the data into a list
  known_pairs = list(map(set,data[['SenderID','ReceiverID']].values))
  len_known_pairs = len(known_pairs)


  #Manually checking if the pair exists
  arr = []
  for i in possible_pairs:
    found = False
    for j in known_pairs:
      if i==j:
        found = True
        break

    if found:
      arr.append(True)
    else:
      arr.append(False)

  arr = np.array(arr)

  #Filtering the leftover pairs and randomly choosing from them an equal number of negatives as positives
  leftover_pairs = pd.DataFrame(possible_pairs, columns=['SenderID','ReceiverID'])[~arr]
  rng.shuffle(leftover_pairs.values)
  chosen = leftover_pairs[:len_known_pairs].reset_index(drop=True)

  #Creating a deep copy and updating the ID pairs to the random leftovers chosen
  copy = data.copy(deep=True).reset_index(drop=True)
  copy["SenderID"], copy["ReceiverID"], copy["present"] = chosen["SenderID"], chosen["ReceiverID"], 0
  # print(copy[['SenderID','ReceiverID','present']], 'Length:', len(copy), len_known_pairs)

  if negative_only:
    return copy

  else:
    results = pd.concat((data.copy(deep=True), copy), axis=0).values.tolist()
    rng.shuffle(results)
    results = pd.DataFrame(results, columns = data.columns)
    return results


# Model Building

In [12]:
#Device Agnostic Code
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


## Data Prep

In [13]:
def pad_input(data, size=50):
  '''
  size: the length that the data will be padded/shortened to; defaults to 50
  '''
  # print(data.shape[2])
  features = np.zeros((1, size, data.shape[2]),dtype=float)
  for ii, row in enumerate(data):
      if len(row) != 0:
          features[ii, -len(row):] = np.array(row)[:size]
  return features



### Train test split according to timestamps

In [14]:
# Getting all unique timestamps
timestamps = np.unique(scaled_data['Timestamp'])
timestamps, timestamps.shape

(array([-2.10915175, -2.10761067, -2.1060696 , ...,  2.06099688,
         2.06356538,  2.06407907]),
 (2233,))

In [15]:
# Splitting the unique timestamps to a 8:2 ratio
train_size = 0.8
train_timestamps = timestamps[:round(len(timestamps)*train_size)]
test_timestamps = timestamps[round(len(timestamps)*train_size): ]
train_timestamps.shape, test_timestamps.shape

((1786,), (447,))

In [16]:
# Getting max_size for pad_input function
shapes = []
for timestamp in timestamps:
  current = scaled_data[scaled_data['Timestamp'] == timestamp]
  shapes.append(current.shape[0])

max_size = max(shapes)*2 # x2 because we have to make equal number negative as positives
max_size

150

Train data

In [17]:
timestamp_train_data = scaled_data[np.isin(scaled_data['Timestamp'], train_timestamps)]
timestamp_train_data['present'] = 1
timestamp_train_data

<ipython-input-17-38253897283d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  timestamp_train_data['present'] = 1


,SenderID,ReceiverID,Receiver_XVelocity,Timestamp,Pathloss,PropDelay,Distance,Sender_XVelocity,Sender_YVelocity,Receiver_YVelocity,present
0,1,2,-0.415269,-2.109152,-1.272443,-1.232738,-1.232222,-0.575348,-0.725865,-0.550458,1
1,1,3,-0.415269,-2.109152,-1.427426,-1.264487,-1.264923,-0.575348,-0.725865,-0.550458,1
2,1,4,-0.415269,-2.109152,-1.175571,-1.207890,-1.208487,-0.575348,-0.725865,-0.550458,1
3,1,5,-0.415269,-2.109152,-1.351286,-1.249303,-1.249612,-0.575348,-0.725865,-0.550458,1
4,1,6,-0.415269,-2.109152,-0.963413,-1.145772,-1.145856,-0.575348,-0.725865,-0.550458,1
...,...,...,...,...,...,...,...,...,...,...,...
39803,1,22,-0.415269,0.925225,0.764216,0.773003,0.773631,1.035886,1.005340,-0.550458,1
39804,1,23,-0.415269,0.925225,0.713556,0.644625,0.644631,1.035886,1.005340,-0.550458,1
39805,1,24,-0.415269,0.925225,0.800216,0.869632,0.870132,1.035886,1.005340,-0.550458,1
39806,1,25,-0.415269,0.925225,0.747799,0.731591,0.730976,1.035886,1.005340,-0.550458,1


In [18]:
new_timestamp_train_data = pd.DataFrame([], columns= timestamp_train_data.columns)
for timestamp in train_timestamps:
  current_timestamp = timestamp_train_data[timestamp_train_data['Timestamp'] == timestamp].copy(deep=True).reset_index(drop=True)
  new_timestamp_train_data = pd.concat((new_timestamp_train_data,  make_negatives(possible_pairs, current_timestamp, random_state=42, negative_only=False)), axis=0)

new_timestamp_train_data

,SenderID,ReceiverID,Receiver_XVelocity,Timestamp,Pathloss,PropDelay,Distance,Sender_XVelocity,Sender_YVelocity,Receiver_YVelocity,present
0,1.0,25.0,-0.415269,-2.109152,0.944920,1.303082,1.302749,-0.575348,-0.725865,-0.550458,1.0
1,11.0,23.0,-0.415269,-2.109152,0.916082,1.210594,1.210455,-0.575348,-0.725865,-0.550458,0.0
2,25.0,14.0,-0.415269,-2.109152,0.919353,1.220257,1.220794,-0.575348,-0.725865,-0.550458,0.0
3,1.0,21.0,-0.415269,-2.109152,0.919353,1.220257,1.220794,-0.575348,-0.725865,-0.550458,1.0
4,9.0,2.0,-0.415269,-2.109152,0.417217,0.027580,0.028075,-0.575348,-0.725865,-0.550458,0.0
...,...,...,...,...,...,...,...,...,...,...,...
45,19.0,7.0,1.462951,0.925225,-2.219772,-1.363877,-1.363385,1.035886,1.005340,1.418618,0.0
46,1.0,13.0,-0.416087,0.925225,-0.132684,-0.670909,-0.671405,1.035886,1.005340,-0.544336,1.0
47,1.0,4.0,1.462951,0.925225,-1.175571,-1.207890,-1.208487,1.035886,1.005340,1.418618,1.0
48,1.0,17.0,-0.416087,0.925225,-0.186442,-0.717843,-0.718270,1.035886,1.005340,-0.544336,1.0


In [90]:
def make_formatted_tensor(timestamps, data, max_size=max_size, lookback=5):
  X = {0:[],1:[],2:[],3:[],4:[]}
  y = {0:[],1:[],2:[],3:[],4:[]}

  # Seperating the data according to the timestamp
  for i in range(len(timestamps)-lookback):

    window = timestamps[i:i+lookback]

    for i2, ts in enumerate(window):
      current = data[data['Timestamp'] == ts].values

      # Reshaping the feature and target
      feature = torch.from_numpy(current[:,:-1]).unsqueeze(dim=0)
      target = torch.from_numpy(current[:,-1]).unsqueeze(dim=0).unsqueeze(dim=2)

      # Padding
      feature = torch.Tensor(pad_input(feature,max_size))
      target = torch.Tensor(pad_input(target,max_size))

      if i==0:
        X[i2]= feature
        y[i2]= target

      else:
        X[i2] = torch.cat((X[i2], feature))
        y[i2] = torch.cat((y[i2], target))

  # Converting dictionary to Tensor
  for i3 in range(len(X)):
    col_X = X[i3].unsqueeze(dim=0)
    col_y = y[i3].unsqueeze(dim=0)
    if i3 == 0:
      new_X, new_y = col_X, col_y
      continue

    new_X = torch.cat((new_X,col_X))
    new_y = torch.cat((new_y,col_y))

  # Reshapping the tensor
  new_X = new_X.reshape(len(X),len(X[0]),-1)
  new_X = torch.transpose(new_X, 0,1)

  new_y = new_y.reshape(len(y),len(y[0]),-1)
  new_y = torch.transpose(new_y, 0,1)

  return new_X, new_y

In [91]:
new_X_train, new_y_train = make_formatted_tensor(train_timestamps, new_timestamp_train_data)
display(new_X_train)
new_X_train.shape

tensor([[[ 0.0000,  0.0000,  0.0000,  ..., -0.5753, -0.7259, -0.5505],
         [ 0.0000,  0.0000,  0.0000,  ..., -0.5753, -0.7259, -0.5505],
         [ 0.0000,  0.0000,  0.0000,  ..., -0.5753, -0.7259, -0.5505],
         [ 0.0000,  0.0000,  0.0000,  ..., -0.5753, -0.7259, -0.5505],
         [ 0.0000,  0.0000,  0.0000,  ..., -0.5753, -0.7259, -0.5505]],

        [[ 0.0000,  0.0000,  0.0000,  ..., -0.5753, -0.7259, -0.5505],
         [ 0.0000,  0.0000,  0.0000,  ..., -0.5753, -0.7259, -0.5505],
         [ 0.0000,  0.0000,  0.0000,  ..., -0.5753, -0.7259, -0.5505],
         [ 0.0000,  0.0000,  0.0000,  ..., -0.5753, -0.7259, -0.5505],
         [ 0.0000,  0.0000,  0.0000,  ..., -0.5753, -0.7259, -0.5505]],

        [[ 0.0000,  0.0000,  0.0000,  ..., -0.5753, -0.7259, -0.5505],
         [ 0.0000,  0.0000,  0.0000,  ..., -0.5753, -0.7259, -0.5505],
         [ 0.0000,  0.0000,  0.0000,  ..., -0.5753, -0.7259, -0.5505],
         [ 0.0000,  0.0000,  0.0000,  ..., -0.5753, -0.7259, -0.5505],
  

torch.Size([1781, 5, 1500])

In [92]:
display(new_y_train)
new_y_train.shape

tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 1., 0., 0.],
         [0., 0., 0.,  ..., 1., 0., 1.],
         [0., 0., 0.,  ..., 1., 1., 0.],
         [0., 0., 0.,  ..., 0., 1., 1.]],

        [[0., 0., 0.,  ..., 1., 0., 0.],
         [0., 0., 0.,  ..., 1., 0., 1.],
         [0., 0., 0.,  ..., 1., 1., 0.],
         [0., 0., 0.,  ..., 0., 1., 1.],
         [0., 0., 0.,  ..., 1., 1., 0.]],

        [[0., 0., 0.,  ..., 1., 0., 1.],
         [0., 0., 0.,  ..., 1., 1., 0.],
         [0., 0., 0.,  ..., 0., 1., 1.],
         [0., 0., 0.,  ..., 1., 1., 0.],
         [0., 0., 0.,  ..., 0., 1., 0.]],

        ...,

        [[0., 0., 0.,  ..., 1., 0., 1.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 1., 1., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 1., 1., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0.,

torch.Size([1781, 5, 150])

In [93]:
'''
train_data_seq = None
flag = True
# Seperating the data according to the timestamp
for timestamp in train_timestamps:
  current = new_timestamp_train_data[new_timestamp_train_data['Timestamp'] == timestamp].values.tolist()
  current = torch.Tensor(current).unsqueeze(dim=0)
  current = pad_input(current, max_size) #<-- this should be the max, not an arbitarily put number <- will cause the dropout rate to be very high
  current = torch.Tensor(current).to(device)

  if flag:
    flag = False
    train_data_seq = current
    continue

  train_data_seq = torch.cat((train_data_seq, current), dim=0)

train_data_seq

'''

"\ntrain_data_seq = None\nflag = True\n# Seperating the data according to the timestamp\nfor timestamp in train_timestamps:\n  current = new_timestamp_train_data[new_timestamp_train_data['Timestamp'] == timestamp].values.tolist()\n  current = torch.Tensor(current).unsqueeze(dim=0)\n  current = pad_input(current, max_size) #<-- this should be the max, not an arbitarily put number <- will cause the dropout rate to be very high\n  current = torch.Tensor(current).to(device)\n\n  if flag:\n    flag = False\n    train_data_seq = current\n    continue\n\n  train_data_seq = torch.cat((train_data_seq, current), dim=0)\n\ntrain_data_seq\n\n"

In [20]:
# train_data_seq.shape

torch.Size([1786, 150, 11])

Test data

In [21]:
timestamp_test_data = scaled_data[np.isin(scaled_data['Timestamp'], test_timestamps)]
timestamp_test_data['present'] = 1
timestamp_test_data

<ipython-input-21-588a8fc2f218>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  timestamp_test_data['present'] = 1


,SenderID,ReceiverID,Receiver_XVelocity,Timestamp,Pathloss,PropDelay,Distance,Sender_XVelocity,Sender_YVelocity,Receiver_YVelocity,present
39808,14,1,1.462951,0.927279,-0.197696,-0.727506,-0.727710,-0.576049,-0.720483,1.418618,1
39809,14,2,1.462951,0.927279,-0.343771,-0.839320,-0.839366,-0.576049,-0.720483,1.418618,1
39810,14,3,1.462951,0.927279,-0.406187,-0.882113,-0.881455,-0.576049,-0.720483,1.418618,1
39811,14,4,1.462951,0.927279,-0.434967,-0.900058,-0.899837,-0.576049,-0.720483,1.418618,1
39812,14,5,1.462951,0.927279,-0.341792,-0.837939,-0.837984,-0.576049,-0.720483,1.418618,1
...,...,...,...,...,...,...,...,...,...,...,...
49995,14,17,-0.539460,2.064079,-1.550183,-1.286574,-1.286861,-0.681885,0.091776,0.379528,1
49996,14,18,-0.539460,2.064079,-1.973399,-1.341790,-1.342003,-0.681885,0.091776,0.379528,1
49997,14,19,-0.415269,2.064079,0.474808,0.131111,0.131483,-0.681885,0.091776,-0.550458,1
49998,14,20,-0.415269,2.064079,0.405394,0.008254,0.007707,-0.681885,0.091776,-0.550458,1


In [95]:
new_X_test, new_y_test = make_formatted_tensor(test_timestamps, timestamp_test_data)
display(new_X_test)
new_X_test.shape

tensor([[[ 0.0000,  0.0000,  0.0000,  ..., -0.5760, -0.7205,  1.4186],
         [ 0.0000,  0.0000,  0.0000,  ..., -0.5760, -0.7205,  1.4186],
         [ 0.0000,  0.0000,  0.0000,  ..., -0.5753, -0.7259,  1.4186],
         [ 0.0000,  0.0000,  0.0000,  ..., -0.5760, -0.7205,  1.4186],
         [ 0.0000,  0.0000,  0.0000,  ..., -0.5760, -0.7205,  1.4186]],

        [[ 0.0000,  0.0000,  0.0000,  ..., -0.5760, -0.7205,  1.4186],
         [ 0.0000,  0.0000,  0.0000,  ..., -0.5753, -0.7259,  1.4186],
         [ 0.0000,  0.0000,  0.0000,  ..., -0.5760, -0.7205,  1.4186],
         [ 0.0000,  0.0000,  0.0000,  ..., -0.5760, -0.7205,  1.4186],
         [ 0.0000,  0.0000,  0.0000,  ..., -0.5760, -0.7205,  1.4186]],

        [[ 0.0000,  0.0000,  0.0000,  ..., -0.5753, -0.7259,  1.4186],
         [ 0.0000,  0.0000,  0.0000,  ..., -0.5760, -0.7205,  1.4186],
         [ 0.0000,  0.0000,  0.0000,  ..., -0.5760, -0.7205,  1.4186],
         [ 0.0000,  0.0000,  0.0000,  ..., -0.5760, -0.7205,  1.4186],
  

torch.Size([442, 5, 1500])

In [94]:
'''
test_data_seq = None
flag = True
# Seperating the data according to the timestamp
for timestamp in test_timestamps:
  current = timestamp_test_data[timestamp_test_data['Timestamp'] == timestamp].values.tolist()
  current = torch.Tensor(current).unsqueeze(dim=0)
  current = pad_input(current, max_size)
  current = torch.Tensor(current).to(device)

  if flag:
    flag = False
    test_data_seq = current
    continue

  test_data_seq = torch.cat((test_data_seq, current), dim=0)

test_data_seq
'''

"\ntest_data_seq = None\nflag = True\n# Seperating the data according to the timestamp\nfor timestamp in test_timestamps:\n  current = timestamp_test_data[timestamp_test_data['Timestamp'] == timestamp].values.tolist()\n  current = torch.Tensor(current).unsqueeze(dim=0)\n  current = pad_input(current, max_size)\n  current = torch.Tensor(current).to(device)\n\n  if flag:\n    flag = False\n    test_data_seq = current\n    continue\n\n  test_data_seq = torch.cat((test_data_seq, current), dim=0)\n\ntest_data_seq\n"

In [23]:
# test_data_seq.shape

torch.Size([447, 150, 11])

### Creating final dataset with lookback windows

In [96]:
'''
def create_dataset(dataset, lookback):
    X, y = None, None
    for i in range(len(dataset)-lookback):
        feature = dataset[:,:,:-1][i:i+lookback]
        target = dataset[:,:,-1].unsqueeze(dim=2)[i+1:i+lookback+1]
        if i == 0:
          X,y  = feature, target
          continue

        X = torch.cat((X,feature), dim=0)
        y = torch.cat((y,target), dim=0)
    return X,y

# Seperate the data such that each tensor contains 5 timestamps
lookback = 5
X_train, y_train = create_dataset(train_data_seq, lookback=lookback)
X_test, y_test = create_dataset(test_data_seq, lookback=lookback)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)
'''

'\ndef create_dataset(dataset, lookback):\n    X, y = None, None\n    for i in range(len(dataset)-lookback):\n        feature = dataset[:,:,:-1][i:i+lookback]\n        target = dataset[:,:,-1].unsqueeze(dim=2)[i+1:i+lookback+1]\n        if i == 0:\n          X,y  = feature, target\n          continue\n\n        X = torch.cat((X,feature), dim=0)\n        y = torch.cat((y,target), dim=0)\n    return X,y\n\n# Seperate the data such that each tensor contains 5 timestamps\nlookback = 5\nX_train, y_train = create_dataset(train_data_seq, lookback=lookback)\nX_test, y_test = create_dataset(test_data_seq, lookback=lookback)\nprint(X_train.shape, y_train.shape)\nprint(X_test.shape, y_test.shape)\n'

In [25]:
# X_train[0].shape

torch.Size([150, 10])

In [26]:
# X_train[0]

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [16.0000,  3.0000, -0.4153,  ..., -0.5753, -0.7259, -0.5505],
        [16.0000,  7.0000, -0.4153,  ..., -0.5753, -0.7259, -0.5505],
        [18.0000, 14.0000, -0.4153,  ..., -0.5753, -0.7259, -0.5505]],
       device='cuda:0')

## Model

In [97]:
#Hyper Parameters
input_size = new_X_train.shape[-1]
num_layers = 1
sequence_length = 5

hidden_size = 128
#num_classes = 1

num_epochs = 600 #1000
batch_size = 64
learning_rate = 0.0001 #0.001

In [98]:
torch.manual_seed(42)

In [103]:
class LSTMEncoder(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers):
    super(LSTMEncoder, self).__init__()
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.num_layers = num_layers

    self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
    self.encoder = nn.Embedding(batch_size*128,128)


  def forward(self, x):
      out, (hn,cn) = self.lstm(x) # x -> (batch_size, seq, input_size)
      print(hn.shape)
      print(hn)
      # out: (batch_size, seq_length, hidden_size) # from rnn
      # out(N, 5, 5)
      enc = self.encoder(hn)
      return enc

model = LSTMEncoder(input_size, hidden_size, num_layers).to(device)

#loss and optimizer
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(new_X_train,new_y_train), batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(new_X_test,new_y_test), batch_size=batch_size)

#Training Loop V3
model.train()

n_total_steps = len(train_loader)
for epoch in range(1):
  for i, (x_batch,y_batch) in enumerate(train_loader):
    x_batch, y_batch = x_batch.to(device), y_batch.to(device)

    #forward
    # print('x_batch before reshaping: ',x_batch.shape)
    # x_batch = x_batch.reshape(-1, sequence_length, input_size).to(device)
    # print('x_batch after reshaping: ', x_batch.shape)
    enc = model(x_batch)
    print(enc)

    # y_batch = y_batch.squeeze(dim=1)[:,-1]
    # print(outputs,y_batch,sep='\n')
    # print(outputs.shape, y_batch.shape)

    # loss = criterion(outputs, y_batch)

    #backward
    # optimizer.zero_grad()
    # loss.backward()
    # optimizer.step()

    print(f'Epoch {epoch+1}/{num_epochs}, step {i+1}/{n_total_steps}, loss = {loss.item()}')

    break

torch.Size([1, 64, 128])
tensor([[[ 3.5301e-05,  2.3121e-03,  5.4087e-01,  ..., -4.8510e-03,
          -9.9361e-01,  1.2531e-04],
         [-1.3170e-03,  1.2633e-01,  4.4611e-03,  ..., -2.3450e-03,
          -9.6554e-01,  3.1675e-06],
         [ 1.7094e-04,  3.0787e-02,  1.5613e-01,  ..., -3.1735e-05,
          -9.9367e-01,  3.9922e-07],
         ...,
         [-4.0147e-01, -5.1991e-05,  2.3920e-01,  ..., -2.3793e-02,
          -6.1471e-02,  5.7809e-03],
         [-4.8409e-02,  5.4830e-06,  2.0616e-01,  ..., -4.3345e-03,
          -4.1354e-01,  3.3980e-06],
         [ 2.6016e-01,  1.7068e-06, -5.7902e-01,  ..., -5.3376e-03,
          -1.8606e-02,  2.2075e-02]]], device='cuda:0',
       grad_fn=<CudnnRnnBackward0>)


RuntimeError: ignored

In [ ]:
class RNN(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers):
    super(RNN, self).__init__()
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.num_layers = num_layers

    self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
    self.fc = nn.Linear(hidden_size,1)

    self.s1 = nn.Sigmoid()


  def forward(self, x):
      out,_ = self.lstm(x) # x -> (batch_size, seq, input_size)
      # out: (batch_size, seq_length, hidden_size) # from rnn
      # out(N, 5, 5)
      out = out[:, -1, :]
      out = self.fc(out)
      out = self.s1(out)
      return out


In [ ]:
model = RNN(input_size, hidden_size, num_layers).to(device)

#loss and optimizer
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X_train,y_train), batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X_test,y_test), batch_size=batch_size)

## Training

In [ ]:
#Training Loop V3
model.train()

n_total_steps = len(train_loader)
for epoch in range(num_epochs):
  for i, (x_batch,y_batch) in enumerate(train_loader):

    #forward
    # print('x_batch before reshaping: ',x_batch.shape)
    x_batch = x_batch.reshape(-1, sequence_length, input_size).to(device)
    # print('x_batch after reshaping: ', x_batch.shape)
    outputs = model(x_batch)

    y_batch = y_batch.squeeze(dim=1)[:,-1]
    # print(outputs,y_batch,sep='\n')
    # print(outputs.shape, y_batch.shape)
    loss = criterion(outputs, y_batch)

    #backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if (epoch+1)%(num_epochs//10) == 0:
      if (i+1)%(n_total_steps//5)==0:
        print(f'Epoch {epoch+1}/{num_epochs}, step {i+1}/{n_total_steps}, loss = {loss.item()}')

      if (i+1)==n_total_steps:
        print('')

Epoch 50/500, step 28/140, loss = 0.13984352350234985
Epoch 50/500, step 56/140, loss = 0.09068052470684052
Epoch 50/500, step 84/140, loss = 0.09432001411914825
Epoch 50/500, step 112/140, loss = 0.058462969958782196
Epoch 50/500, step 140/140, loss = 0.10584773868322372

Epoch 100/500, step 28/140, loss = 0.025381287559866905
Epoch 100/500, step 56/140, loss = 0.014111246913671494
Epoch 100/500, step 84/140, loss = 0.009329600259661674
Epoch 100/500, step 112/140, loss = 0.012124285101890564
Epoch 100/500, step 140/140, loss = 0.016219327226281166

Epoch 150/500, step 28/140, loss = 0.0011823135428130627
Epoch 150/500, step 56/140, loss = 0.00039496051613241434
Epoch 150/500, step 84/140, loss = 0.0003481428138911724
Epoch 150/500, step 112/140, loss = 0.0005873310728929937
Epoch 150/500, step 140/140, loss = 0.0008521536365151405

Epoch 200/500, step 28/140, loss = 3.972095510107465e-05
Epoch 200/500, step 56/140, loss = 1.2683601198659744e-05
Epoch 200/500, step 84/140, loss = 1.51

In [ ]:
#Training Loop V2
'''
model.train()
num_timstamps = len(train_timestamps)
for tsi, time in enumerate(train_timestamps): #tsi is timestamp index, time is the actual value of the timestamp
  # Get data for current timestamp
  current_train_seq = train_data_seq[tsi]

  # Split to training features/ nodes and target ("present")
  X_train, y_train = current_train_seq[:,:,:-1].clone().detach(), current_train_seq[:,:,-1].clone().detach()

  # Creating trainset and train_loader
  trainset = dataset(X_train,y_train)
  train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size)
  n_total_steps = len(train_loader)

  for epoch in range(num_epochs):
    for i, (x_train,y_train) in enumerate(train_loader):
      #---Forward---
      print(x_train.shape)
      # x_train = x_train.reshape(1,sequence_length, -1)
      outputs = model(x_train)
      y_train = y_train[:,-1].unsqueeze(dim=1)

      # print(outputs,y_train,sep='\n')
      # print(outputs.shape, y_train.shape)

      loss = criterion(outputs, y_train)

      #---Backward---
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      # Printing progress
      if (tsi+1)%100 == 0:
        if (epoch+1)%100 == 0:
          print(f'Timestamp {tsi+1}/{num_timstamps}, Epoch {epoch+1}/{num_epochs}, step {i+1}/{n_total_steps}, loss = {loss.item()}')

      if (epoch+1)==num_epochs:
        print('')
'''

'\nmodel.train()\nnum_timstamps = len(train_timestamps)\nfor tsi, time in enumerate(train_timestamps): #tsi is timestamp index, time is the actual value of the timestamp\n  # Get data for current timestamp\n  current_train_seq = train_data_seq[tsi]\n\n  # Split to training features/ nodes and target ("present")\n  X_train, y_train = current_train_seq[:,:,:-1].clone().detach(), current_train_seq[:,:,-1].clone().detach()\n\n  # Creating trainset and train_loader\n  trainset = dataset(X_train,y_train)\n  train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size)\n  n_total_steps = len(train_loader)\n\n  for epoch in range(num_epochs):\n    for i, (x_train,y_train) in enumerate(train_loader):\n      #---Forward---\n      print(x_train.shape)\n      # x_train = x_train.reshape(1,sequence_length, -1)\n      outputs = model(x_train)\n      y_train = y_train[:,-1].unsqueeze(dim=1)\n\n      # print(outputs,y_train,sep=\'\n\')\n      # print(outputs.shape, y_train.shape)\n\n    

In [ ]:
# #Training Loop
# model.train()

# n_total_steps = len(train_loader)
# for epoch in range(num_epochs):
#   for i, (x_train,y_train) in enumerate(train_loader):
#     # print(f'Iter {i+1}, x_train:{x_train[:5]}, y_train:{y_train[:5]}')

#     #forward
#     # print('x_train before reshaping: ',x_train.shape)
#     x_train = x_train.reshape(-1, sequence_length, input_size).to(device)
#     # print('x_train after reshaping: ', x_train.shape)
#     outputs = model(x_train)

#     y_train = y_train[:,-1].unsqueeze(dim=1)

#     # print(outputs,y_train,sep='\n')
#     # print(outputs.shape, y_train.shape)
#     loss = criterion(outputs, y_train)

#     #backward
#     optimizer.zero_grad()
#     loss.backward()
#     optimizer.step()
#     if (epoch+1)%100 == 0:
#       print(f'Epoch {epoch+1}/{num_epochs}, step {i+1}/{n_total_steps}, loss = {loss.item():.5f}')

#       if (i+1)==n_total_steps:
#         print('')

## Testing

In [ ]:
model.input_size, model.num_layers, model.hidden_size

(100, 1, 128)

In [ ]:
def binary_threshold(x, threshold= 0.5):
  # print(x)
  threshold = torch.tensor([threshold]).to(device)
  results = (x>threshold).float()*1
  # print(results)
  return results

In [ ]:
#Testing Loop for Training Data
model.eval()
n_correct = 0
n_samples = 0
with torch.inference_mode():
  for i, (x_batch,y_batch) in enumerate(train_loader):
    x_batch = x_batch.reshape(-1, sequence_length, input_size).to(device)
    y_batch = y_batch.squeeze(dim=1)[:,-1].to(device)
    outputs = model(x_batch)
    outputs = binary_threshold(outputs)

    # print(outputs.squeeze(),y_batch.squeeze())
    # print(outputs.shape, y_batch.shape)

    n_samples += y_batch.shape[0] * y_batch.shape[1]

    n_correct += (outputs == y_batch).sum().item()
    # print((outputs==y_batch).sum())
    # print(n_correct, n_samples)

acc = 100* n_correct/n_samples
print(f'Training Accuracy = {acc:.3f}%')

Training Accuracy = 100.000%


In [ ]:
#Testing Loop for Testing Data
model.eval()
n_correct = 0
n_samples = 0
with torch.inference_mode():
  for i, (x_batch,y_batch) in enumerate(test_loader):
    x_batch = x_batch.reshape(-1, sequence_length, input_size).to(device)
    y_batch = y_batch.squeeze(dim=1)[:,-1].to(device)
    outputs = model(x_batch)
    # print(outputs)
    outputs = binary_threshold(outputs)

    # print(outputs,y_batch)
    # print(outputs.shape, y_batch.shape)

    ### If we only use the new unseen data
    outputs = outputs[:,-1]
    y_batch = y_batch[:,-1]
    # print(y_batch)

    # print(outputs,y_batch)
    # print(outputs.shape, y_batch.shape)

    # n_samples += y_batch.shape[0] * y_batch.shape[1]
    n_samples += y_batch.shape[0]

    n_correct += (outputs == y_batch).sum().item()

    print(f'Cumulative: {n_correct}/{n_samples}')

acc = 100* n_correct/n_samples
print(f'Testing Accuracy = {acc:.3f}%')

Cumulative: 49/64
Cumulative: 76/128
Cumulative: 97/192
Cumulative: 146/256
Cumulative: 193/320
Cumulative: 228/384
Cumulative: 251/448
Cumulative: 302/512
Cumulative: 330/576
Cumulative: 353/640
Cumulative: 396/704
Cumulative: 419/768
Cumulative: 470/832
Cumulative: 517/896
Cumulative: 543/960
Cumulative: 588/1024
Cumulative: 622/1088
Cumulative: 665/1152
Cumulative: 719/1216
Cumulative: 748/1280
Cumulative: 801/1344
Cumulative: 847/1408
Cumulative: 887/1472
Cumulative: 931/1536
Cumulative: 982/1600
Cumulative: 1019/1664
Cumulative: 1062/1728
Cumulative: 1119/1792
Cumulative: 1162/1856
Cumulative: 1207/1920
Cumulative: 1252/1984
Cumulative: 1287/2048
Cumulative: 1339/2112
Cumulative: 1393/2176
Cumulative: 1412/2210
Testing Accuracy = 63.891%
